**Necessary package installs for everything to be functional.**

In [ ]:
# First, install the stripe library (uncomment the next line if not installed)
!pip install stripe
!pip install stripe ipywidgets
!pip install stripe ipywidgets qrcode[pil]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 396.6 kB/s eta 0:00:00


**Necessary Imports for stripe and some webinteraction for front end.**

In [ ]:
import stripe
import ipywidgets as widgets
from IPython.display import display, HTML, Image
import qrcode
import io

stripe.api_key = "sk_test_51QtZf905Lo986fYCauDB7uTHm0jtKqdNrYC4HkOl8n1CLtdkA5ZJVNF1isK15PqFObQ9Ax8IQutOsgmJRWBla76400Y5BxJB2I"

# **Process Payment with pre-defined card. (it is a test-card)**

In [ ]:
def process_payment():
    """
    Processes a payment by creating a PaymentIntent using a test token for a Visa card.
    """
    try:
        payment_intent = stripe.PaymentIntent.create(
            amount=1099,
            currency="usd",
            payment_method_data={
                "type": "card",
                "card": {
                    "token": "tok_visa"
                }
            },
            confirm=True,
            return_url="https://example.com/return",
            description="Test Payment using Visa test token with return_url"
        )
        print("PaymentIntent created and confirmed successfully:")
        print(payment_intent)
    except Exception as e:
        print("An error occurred while processing the payment:", e)


In [ ]:

# Create a button widget with the label "Pay"
pay_button = widgets.Button(
    description="Pay",
    button_style='primary',
    tooltip="Click to process the payment"
)
# Create an output widget to display the payment response or errors
output = widgets.Output()
def on_pay_button_clicked(b):
    with output:
        output.clear_output()
        process_payment()
pay_button.on_click(on_pay_button_clicked)
display(pay_button, output)

Button(button_style='primary', description='Pay', style=ButtonStyle(), tooltip='Click to process the payment')

Output()

# **Using stripe checkout.**

## **No QR code**

In [ ]:
def create_checkout_session():
    """
    Creates a Stripe Checkout session to collect card details securely.
    """
    try:
        session = stripe.checkout.Session.create(
            payment_method_types=['card'],
            line_items=[
                {
                    'price_data': {
                        'currency': 'usd',
                        'product_data': {
                            'name': 'Test Payment',
                        },
                        'unit_amount': 1099,
                    },
                    'quantity': 1,
                },
            ],
            mode='payment',
            success_url='https://example.com/success?session_id={CHECKOUT_SESSION_ID}',
            cancel_url='https://example.com/cancel',
        return session
    except Exception as e:
        print("An error occurred while creating checkout session:", e)
        return None


In [ ]:
checkout_button = widgets.Button(
    description="Pay",
    button_style='primary',
    tooltip="Click to open the Stripe Checkout page"
)
output = widgets.Output()

def on_checkout_button_clicked(b):
    with output:
        output.clear_output()
        session = create_checkout_session()
        if session:
            print("Checkout session created successfully!")
            print("Please complete your payment on Stripe's secure Checkout page:")
            # Display a clickable link to the Checkout session URL
            display(HTML(f'<a href="{session.url}" target="_blank">Click here to pay</a>'))
        else:
            print("Failed to create checkout session.")
checkout_button.on_click(on_checkout_button_clicked)
display(checkout_button, output)


Button(button_style='primary', description='Pay', style=ButtonStyle(), tooltip='Click to open the Stripe Check…

Output()

## With QR code

In [ ]:

def create_checkout_session():
    """
    Creates a Stripe Checkout session to collect card details securely.
    """
    try:
        session = stripe.checkout.Session.create(
            payment_method_types=['card'],
            line_items=[
                {
                    'price_data': {
                        'currency': 'usd',
                        'product_data': {
                            'name': 'Test Payment',
                        },
                        'unit_amount': 1099,
                    },
                    'quantity': 1,
                },
            ],
            mode='payment',
            success_url='https://example.com/success?session_id={CHECKOUT_SESSION_ID}',
            cancel_url='https://example.com/cancel',
        )
        return session
    except Exception as e:
        print("An error occurred while creating checkout session:", e)
        return None

checkout_button = widgets.Button(
    description="Pay",
    button_style='primary',
    tooltip="Click to open the Stripe Checkout page"
)

output = widgets.Output()

def on_checkout_button_clicked(b):
    with output:
        output.clear_output()
        session = create_checkout_session()
        if session:
            print("Checkout session created successfully!")
            print("Please complete your payment on Stripe's secure Checkout page by scanning the QR code below:")

            # Generate a QR code from the session URL
            qr = qrcode.QRCode(
                version=1,
                error_correction=qrcode.constants.ERROR_CORRECT_L,
                box_size=10,
                border=4,
            )
            qr.add_data(session.url)
            qr.make(fit=True)
            img = qr.make_image(fill_color="black", back_color="white")

            # Save the image to a BytesIO buffer and display it
            buf = io.BytesIO()
            img.save(buf, format='PNG')
            buf.seek(0)
            display(Image(data=buf.getvalue()))
        else:
            print("Failed to create checkout session.")

checkout_button.on_click(on_checkout_button_clicked)
display(checkout_button, output)

Button(button_style='primary', description='Pay', style=ButtonStyle(), tooltip='Click to open the Stripe Check…

Output()

In [ ]:
import stripe
import qrcode
import io
import time
import threading
from IPython.display import display, Image
import ipywidgets as widgets

def create_checkout_session():
    """
    Creates a Stripe Checkout session to collect card details securely.
    """
    try:
        session = stripe.checkout.Session.create(
            payment_method_types=['card'],
            line_items=[
                {
                    'price_data': {
                        'currency': 'usd',
                        'product_data': {
                            'name': 'Test Payment',
                        },
                        'unit_amount': 1099,
                    },
                    'quantity': 1,
                },
            ],
            mode='payment',
            success_url='https://example.com/success?session_id={CHECKOUT_SESSION_ID}',
            cancel_url='https://example.com/cancel',
        )
        return session
    except Exception as e:
        print("An error occurred while creating checkout session:", e)
        return None

checkout_button = widgets.Button(
    description="Pay",
    button_style='primary',
    tooltip="Click to open the Stripe Checkout page"
)

output = widgets.Output()

def poll_payment_status(session_id):
    # Poll the checkout session every few seconds for payment success
    while True:
        session = stripe.checkout.Session.retrieve(session_id)
        if session.payment_status == 'paid':
            with output:
                print("Payment successful!")
            break
        time.sleep(3)  # poll every 3 seconds

def on_checkout_button_clicked(b):
    with output:
        output.clear_output()
        session = create_checkout_session()
        if session:
            print("Checkout session created successfully!")
            print("Please complete your payment on Stripe's secure Checkout page by scanning the QR code below:")

            # Generate a QR code from the session URL
            qr = qrcode.QRCode(
                version=1,
                error_correction=qrcode.constants.ERROR_CORRECT_L,
                box_size=10,
                border=4,
            )
            qr.add_data(session.url)
            qr.make(fit=True)
            img = qr.make_image(fill_color="black", back_color="white")

            # Save the image to a BytesIO buffer and display it
            buf = io.BytesIO()
            img.save(buf, format='PNG')
            buf.seek(0)
            display(Image(data=buf.getvalue()))

            # Start polling for the payment status in a background thread
            threading.Thread(target=poll_payment_status, args=(session.id,), daemon=True).start()
        else:
            print("Failed to create checkout session.")

checkout_button.on_click(on_checkout_button_clicked)
display(checkout_button, output)


Button(button_style='primary', description='Pay', style=ButtonStyle(), tooltip='Click to open the Stripe Check…

Output()

In [ ]:
import stripe
import qrcode
import io
import asyncio
from IPython.display import display, Image
import ipywidgets as widgets

def create_checkout_session():
    """
    Creates a Stripe Checkout session to collect card details securely.
    """
    try:
        session = stripe.checkout.Session.create(
            payment_method_types=['card'],
            line_items=[
                {
                    'price_data': {
                        'currency': 'usd',
                        'product_data': {
                            'name': 'Test Payment',
                        },
                        'unit_amount': 1099,
                    },
                    'quantity': 1,
                },
            ],
            mode='payment',
            success_url='https://example.com/success?session_id={CHECKOUT_SESSION_ID}',
            cancel_url='https://example.com/cancel',
        )
        return session
    except Exception as e:
        print("An error occurred while creating checkout session:", e)
        return None

checkout_button = widgets.Button(
    description="Pay",
    button_style='primary',
    tooltip="Click to open the Stripe Checkout page"
)

output = widgets.Output()

async def poll_payment_status(session_id):
    while True:
        session = stripe.checkout.Session.retrieve(session_id)
        if session.payment_status == 'paid':
            with output:
                print("Payment successful!")
            break
        await asyncio.sleep(3)  # non-blocking sleep

def on_checkout_button_clicked(b):
    with output:
        output.clear_output()
        session = create_checkout_session()
        if session:
            print("Checkout session created successfully!")
            print("Please complete your payment on Stripe's secure Checkout page by scanning the QR code below:")

            # Generate a QR code from the session URL
            qr = qrcode.QRCode(
                version=1,
                error_correction=qrcode.constants.ERROR_CORRECT_L,
                box_size=10,
                border=4,
            )
            qr.add_data(session.url)
            qr.make(fit=True)
            img = qr.make_image(fill_color="black", back_color="white")

            buf = io.BytesIO()
            img.save(buf, format='PNG')
            buf.seek(0)
            display(Image(data=buf.getvalue()))

            # Schedule the async polling task without blocking
            asyncio.create_task(poll_payment_status(session.id))
        else:
            print("Failed to create checkout session.")

checkout_button.on_click(on_checkout_button_clicked)
display(checkout_button, output)


Button(button_style='primary', description='Pay', style=ButtonStyle(), tooltip='Click to open the Stripe Check…

Output()